In [2]:
import os
import numpy as np
import open3d as o3d
import igl
import meshplot as mp
from matplotlib import pyplot as plt
from matplotlib import cm
import math
import json

In [24]:
"""Functions: for raw .ply input!"""
# input: mesh file (path)
# output1: vertices (N, 3) array; x y z coordinates
# output2: faces (F, 3) array, format: [a id, b id, c id]
def get_mesh(path):
    mesh = o3d.io.read_triangle_mesh(path)
    print(mesh)
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    return verts, faces
# input: output file name, neighbor (k), crestline (1 = yes, 0 = no), vertices (array), faces (array)
# does: txt file <- for CrestCODE
def to_pseudo_PLY2(name, neighbor, verts, faces, crestline=1):
    NEWLINE = '\n'
    verts_count = len(verts)
    faces_count = len(faces)
    with open(name, 'w') as f:
        f.write(str(verts_count) + NEWLINE)
        f.write(str(faces_count) + NEWLINE)
        f.write(str(neighbor) + NEWLINE)
        f.write(str(crestline) + NEWLINE)
        for i in range(verts_count): # all vertices x y z coordinates
            f.write(" ".join(map(str, verts[i])) + NEWLINE)
        for j in range(faces_count): # all faces a b c vertices ids
            f.write(" ".join(map(str, faces[j])) + NEWLINE)
    print(f"Success: {name}")
"""Functions: for CrestCODE output"""
# get data from Ridges.txt OR Ravines.txt
# input: meshfile name, mesh vertices, mesh faces
# output1: crest line vertices (V, 4) array, format: [x coordinate, y coordinate, z coordinate, connected component]
# output2: connected components (N, 3) array, format: [Ridgeness, Sphericalness, Cyclideness]
# output3: crest line edges (E, 3) array, format: [u vertex index, v vertex index, triangle ID]
def ReadCrestLine(filename):
    f = open(filename, 'r')
    V = int(f.readline())
    E = int(f.readline())
    N = int(f.readline()) # num of connected components;
    crestline_vertices = np.zeros(shape = (V, 4)) # crest line vertices
    for i in range(V): 
        line = f.readline()
        crestline_vertices[i] = [float(n) for n in line.split()]
    crestline_connected_cmp = np.zeros(shape = (N, 3)) # index = connected cmp ID
    for j in range(N):
        line = f.readline()
        crestline_connected_cmp[j] = [float(n) for n in line.split()]
    # edges (u,v): [vtx ID of u, vtx ID of v, triangle ID]
    crestline_edges = np.zeros(shape = (E, 3), dtype=int) # index = edge ID
    for k in range(E):
        line = f.readline()
        crestline_edges[k] = [n for n in line.split()]
    return crestline_vertices[:,0:3], crestline_edges

In [33]:
"""Display utility"""
purple = np.array([1.0, 0.1, 1.0]) # purple
turquoise = np.array([0.1, 1.0, 0.1]) # turquoise
blue = np.array([0.1, 0.1, 1.0]) # blue
green = np.array([0.1, 1.0, 0.1]) # green
yellow = np.array([1.0, 1.0, 0.1]) # yellow
red = np.array([1.0, 0.1, 0.1]) # red
black = np.array([0, 0, 0]) # black
def mesh_lines(vertices, faces, plot, color):
    plot.add_lines(vertices[faces[:, 0]], 
                    vertices[faces[:, 1]], 
                    shading={"line_color": color})
    plot.add_lines(vertices[faces[:, 0]], 
                    vertices[faces[:, 2]], 
                    shading={"line_color": color})
    plot.add_lines(vertices[faces[:, 1]], 
                    vertices[faces[:, 2]], 
                    shading={"line_color": color})

In [65]:
"""deform"""
def Handles(mesh_vertices, mesh_faces, affected_vertex_ids, target_positions, iterations):
    mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(mesh_vertices),
                                 o3d.utility.Vector3iVector(mesh_faces))
    # constraint_ids = o3d.utility.IntVector(affected_vertex_ids)
    # constraint_pos = o3d.utility.Vector3dVector(target_positions)
    deform = mesh.deform_as_rigid_as_possible(
        constraint_vertex_indices=o3d.utility.IntVector(affected_vertex_ids), 
        constraint_vertex_positions=o3d.utility.Vector3dVector(target_positions), 
        max_iter=iterations)
    return np.asarray(deform.vertices)

In [76]:
"""flatten"""
def write_TLC_input(filename, restVert, initVert, triangles, handles):
	"""write data (numpy array) as TLC program input format"""
	with open(filename, 'w') as file:
		# rest vertices
		nV, nDim = restVert.shape
		file.write(f'{nV} {nDim}\n')
		np.savetxt(file, restVert)
		# init vertices
		nV, nDim = initVert.shape
		file.write(f'{nV} {nDim}\n')
		np.savetxt(file, initVert)
		# simplex cells <- triangles
		nF, simplex_size = triangles.shape
		file.write(f'{nF} {simplex_size}\n')
		np.savetxt(file, triangles, "%d")
		# handles
		nhdls = handles.size
		file.write(f'{nhdls}\n')
		np.savetxt(file, handles, "%d")
def read_TLC_result(filename):
	"""read TLC program results into a dictionary"""
	result_dict = {}
	with open(filename) as file:
		while True:
			line = file.readline()
			if line.strip() == "":
				break
			head = line.split()
			data_name = head[0]
			# print(f"data_name: {data_name}")
			array_shape = [int(i) for i in head[1:]]
			# print(f"head: {head}")
			# print(file)
			# if head[0] == "energy":
			# 	break
			data = np.loadtxt(file, dtype=np.float64, max_rows=1)
			if len(array_shape) == 1:
				result_dict[data_name] = data
			else:
				result_dict[data_name] = np.reshape(data, array_shape)
	return result_dict


In [101]:
class crestline_mover:
    def __init__(self, mesh_3d_vertices, mesh_faces, 
                 mesh_3d_crestlines_vertices, crestline_edges, 
                 mesh_2d_vertices, color):
        self.mesh_3d_vertices = mesh_3d_vertices
        self.mesh_faces = mesh_faces
        self.mesh_3d_crestlines_vertices = mesh_3d_crestlines_vertices[:,0:3]
        self.mesh_2d_vertices = mesh_2d_vertices
        self.color = color
        self.crestline_edges = crestline_edges
        self.V = mesh_3d_vertices.shape[0]
        self.F = mesh_faces.shape[0]
        self.crestline_V = mesh_3d_crestlines_vertices.shape[0]
        self.crestline_E = crestline_edges.shape[0]
        self.mesh_2d_crestlines_vertices = self.Move_3D_vertices_to_2D() # want to find
    def collinear(self, a, b, c):
        # use matrix rank to determine colinearity (no work, needs exact equal)
        # Reference: https://stackoverflow.com/questions/9608148/python-script-to-determine-if-x-y-coordinates-are-colinear
        # -getting-some-e#:~:text=You%20can%20use%20the%20rank,i.e.%20any%20number%20of%20points).

        # added tolerance
        vector_ab = a - b
        vector_ac = a - c
        cross_product = np.cross(vector_ab, vector_ac)
        edge_length_ab = np.sqrt(vector_ab.dot(vector_ab))
        return np.sqrt(cross_product.dot(cross_product)) < edge_length_ab/10000
    # v can be on line (a, b), (a, c), or (b, c)
    def FindVertexIndex(self, a, b, c, a_index, b_index, c_index, v):
        #print(f"checking colinearity of: {a, b} and {v}") 
        if(self.collinear(a, b, v)): 
            return a_index, b_index
        #print(f"checking colinearity of: {a, c} and {v}") 
        if(self.collinear(a, c, v)): 
            return a_index, c_index
        #print(f"checking colinearity of: {b, c} and {v}") 
        if(self.collinear(b, c, v)): 
            return b_index, c_index
        #print("no colinearity found!")
    # input: mesh vertices a and b; vertex v that lies on edge (a, b)
    # return alpha: from a to v on edge (a, b)
    def getAlpha(self, a, b, v):
        va = a - v
        ab = b - a
        va_length = np.sqrt(va.dot(va))
        ab_length = np.sqrt(ab.dot(ab))
        alpha = va_length / ab_length
        return alpha
    def LinearInterpolate(self):
        augmentation = np.zeros(shape=(self.crestline_V, 3))
        for e in range(self.crestline_E): # consider crestline edge (u, v)
            u_index, v_index, face_index = self.crestline_edges[e]
            #print(f"edge information: {u_index} {v_index} {face_index}")
            if(face_index != -1): # when face_index == -1, there's NO triangle associated w/ crestline edge
                face_vertex_a_index, face_vertex_b_index, face_vertex_c_index = self.mesh_faces[face_index]
                #print(f"checking face: {face_vertex_a_index} {face_vertex_b_index} {face_vertex_c_index}")
                # edge lies on triangle (a, b, c) <- these are MESH vertices
                a = self.mesh_3d_vertices[face_vertex_a_index]
                b = self.mesh_3d_vertices[face_vertex_b_index]
                c = self.mesh_3d_vertices[face_vertex_c_index]
                #print(f"which are: \n{a}, \n{b}, \n{c}")
                
                """for the two vertices in (u, v), do:""" # <- u, v are CRESTLINE vertices
                # u is on edge (u1, u2)
                u = self.mesh_3d_crestlines_vertices[u_index] # u can be on any of the 3 edges of the face
                #print(f"checking crestline vertex: {u} at index {u_index}")
                u1, u2 = self.FindVertexIndex(a, b, c, 
                                              face_vertex_a_index, 
                                              face_vertex_b_index, 
                                              face_vertex_c_index, u)
                alpha_u = self.getAlpha(a=self.mesh_3d_vertices[u1], 
                                        b=self.mesh_3d_vertices[u2], v=u)
                # for mesh vertex index@ u1, write down it's: (index) u2, alpha value
                augmentation[u_index][0] = u1
                augmentation[u_index][1] = u2
                augmentation[u_index][2] = alpha_u

                # v is on edge (v1, v2)
                v = self.mesh_3d_crestlines_vertices[v_index] # u can be on any of the 3 edges of the face
                v1, v2 = self.FindVertexIndex(a, b, c, 
                                              face_vertex_a_index, 
                                              face_vertex_b_index, 
                                              face_vertex_c_index, v)
                alpha_v = self.getAlpha(a=self.mesh_3d_vertices[v1], 
                                        b=self.mesh_3d_vertices[v2], v=v)
                # for mesh vertex index@ v1, write down its: (index) v2, alpha value
                augmentation[v_index][0] = v1
                augmentation[v_index][1] = v2
                augmentation[v_index][2] = alpha_v
        return augmentation
    # input: mesh vertices a, b
    # return: point v, which lies on edge (a, b)
    # assumption: alpha refers to: length of edge (v, a) / lendth of edge (b, a)
    def Recover(self, a, b, alpha):
        v = alpha * (b - a) + a
        return v
    def Move_3D_vertices_to_2D(self):
        augmentation = self.LinearInterpolate()
        crestline_V_2d = np.zeros(shape=(self.mesh_3d_crestlines_vertices.shape))
        for e in range(self.crestline_E): # for all mesh vertices:
            # use 3d crestline vertices + augmentation to find 2d creatline vertices
            u_index, v_index, face_index = self.crestline_edges[e]

            u1_index = augmentation[u_index][0].astype(int)
            u2_index = augmentation[u_index][1].astype(int)
            u_alpha = augmentation[u_index][2]
            u1 = self.mesh_2d_vertices[u1_index]
            u2 = self.mesh_2d_vertices[u2_index]
            u_2d = self.Recover(u1, u2, u_alpha)
            crestline_V_2d[u_index] = u_2d
            # print(f"processing point number {u_index + 1}, alpha: {u_alpha}, output: {u_2d}")
            # print(f"    this vertex lies on edge between {u1} and {u2}")

            v1_index = augmentation[v_index][0].astype(int)
            v2_index = augmentation[v_index][1].astype(int)
            v_alpha = augmentation[v_index][2]
            v1 = self.mesh_2d_vertices[v1_index]
            v2 = self.mesh_2d_vertices[v2_index]
            v_2d = self.Recover(v1, v2, v_alpha)
            crestline_V_2d[v_index] = v_2d 
            # print(f"processing point number {v_index + 1}, alpha: {v_alpha}, output: {v_2d}")
            # print(f"    this vertex lies on edge between {v1} and {v2}")
        return crestline_V_2d

In [18]:
mesh_name = "test_230613.ply"
ply2_name = f"{mesh_name}_ply2.txt"
V, F = get_mesh("test_230613.ply")
to_pseudo_PLY2(name=ply2_name, verts=V, faces=F, neighbor=6)
crestcode_input = f"./setCurvature {ply2_name} output.txt"
print(f"in CrestCODE, enter: {crestcode_input}")

TriangleMesh with 13419 points and 26382 triangles.
Success: test_230613.ply_ply2.txt
in CrestCODE, enter: ./setCurvature test_230613.ply_ply2.txt output.txt


In [32]:
crestline_V_3d, crestline_E = ReadCrestLine("ravines.txt")
plot_3d = mp.plot(v=V, f=F)
plot_3d.add_lines(beginning=crestline_V_3d[crestline_E[:,0]], 
                  ending=crestline_V_3d[crestline_E[:,1]], 
                  shading={"line_color": "black", "line_width": 1000.0})
mesh_lines(vertices=V, faces=F, plot=plot_3d, color="red")


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5431385…

In [119]:
mesh_color = np.zeros(shape=V.shape)
mesh_color[:, :] = yellow
mesh_color[0] = turquoise

handle_ids = [0, 1000, 2000, 3000, 6000]
handle_current_pos = V[handle_ids]
print(f"current handle positions: \n{handle_current_pos}")
handle_color = np.asarray([
    green, blue, black, purple, red
])
handle_pos = np.asarray([
                [1.24, 2.60, 0], # green
                [1.32, 2.32, 0], # blue
                [1.18, 2.18, 0], # black
                [1.70, 2.50, 0], # purple
                [1.50, 2.2, 0]  # red
                ])
print(f"target handle positions: \n{handle_pos}")
arap_before = mp.plot(v=V, 
                        f=F,
                        c=mesh_color)
arap_before.add_points(points=handle_current_pos + np.asarray([0, 0, 0.005]), # <- small points: where the handles refer to
                          c=handle_color,
                          shading={"point_size": 0.1})
arap_before.add_points(points=handle_pos + np.asarray([0, 0, -0.15]), # <- big points: where the small points WILL BE
                          c=handle_color,
                          shading={"point_size": 1})
mesh_lines(vertices=V, faces=F, plot=arap_before, color="red")

V_deform = Handles(
    mesh_vertices=V,
    mesh_faces=F,
    affected_vertex_ids=handle_ids,
    target_positions=handle_pos,
    iterations=10
)

arap_after = mp.plot(v=V_deform, 
                        f=F,
                        c=mesh_color)
arap_after.add_points(points=handle_pos + np.asarray([0, 0, 0.005]), # <- small points: where the handles refer to
                          c=handle_color,
                          shading={"point_size": 0.5})
mesh_lines(vertices=V_deform, faces=F, plot=arap_after, color="red")

current handle positions: 
[[ 1.30028415  2.58257699 -0.2368753 ]
 [ 1.24768305  2.35543323 -0.22349977]
 [ 1.20675421  2.20177722 -0.22293329]
 [ 1.66772246  2.59593844 -0.21469522]
 [ 1.50738573  1.961272   -0.19170189]]
target handle positions: 
[[1.24 2.6  0.  ]
 [1.32 2.32 0.  ]
 [1.18 2.18 0.  ]
 [1.7  2.5  0.  ]
 [1.5  2.2  0.  ]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5431385…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5349544…

In [ ]:
./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_100_v2.txt

In [120]:
TLC_filename = f"{mesh_name}_tlc_v2.txt"
flat_filename = f"{mesh_name}_2d_v2.txt"
write_TLC_input(
    filename=TLC_filename,
    restVert=V,
    initVert=V_deform[:,0:2],
    triangles=F,
    handles=np.asarray(handle_ids))
print(f"the TLC file is: {TLC_filename}")
print(f"in SEA, enter: ./SEA_QN {TLC_filename} solver_options.txt {flat_filename}")

the TLC file is: test_230613.ply_tlc_v2.txt
in SEA, enter: ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_v2.txt


In [150]:
flat_filename_100 = f"{mesh_name}_2d_100_v2.txt"
flat_filename_200 = f"{mesh_name}_2d_200_v2.txt"
flat_filename_300 = f"{mesh_name}_2d_300_v2.txt"
flat_filename_400 = f"{mesh_name}_2d_400_v2.txt"
flat_filename_500 = f"{mesh_name}_2d_500_v2.txt"
flat_filename_600 = f"{mesh_name}_2d_600_v2.txt"
flat_filename_700 = f"{mesh_name}_2d_700_v2.txt"
flat_filename_800 = f"{mesh_name}_2d_800_v2.txt"
flat_filename_900 = f"{mesh_name}_2d_900_v2.txt"
flat_filename_1000 = f"{mesh_name}_2d_1000_v2.txt"
flat_filename_1500 = f"{mesh_name}_2d_1500_v2.txt"
flat_filename_2000 = f"{mesh_name}_2d_2000_v2.txt"

flat_filename_10 = f"{mesh_name}_2d_10_v2.txt"
flat_filename_20 = f"{mesh_name}_2d_20_v2.txt"
flat_filename_30 = f"{mesh_name}_2d_30_v2.txt"
flat_filename_40 = f"{mesh_name}_2d_40_v2.txt"
flat_filename_50 = f"{mesh_name}_2d_50_v2.txt"
flat_filename_60 = f"{mesh_name}_2d_60_v2.txt"
flat_filename_70 = f"{mesh_name}_2d_70_v2.txt"
flat_filename_80 = f"{mesh_name}_2d_80_v2.txt"
flat_filename_90 = f"{mesh_name}_2d_90_v2.txt"

In [ ]:
# version 1 result
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt pyramd_result_100.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 51805267 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 100 iterations
# 149 function/gradient evaluations
# TLC time: 23.3184 seconds.
# arc segment time: 0.0926496 seconds.
# arc occupancy time: 25.0998 seconds.
# - arc arrangement time: 24.6001 seconds.
# ** subdivide polyArc time: 24.2879 seconds.
# ### arc intersection init time: 0.0648048 seconds.
# ### arc bbox intersection test time: 4.43916 seconds.
# ### arc-arc intersection time: 19.285 seconds.
# ### arc subdivision time: 0.0188485 seconds.
# ** decompose into cells time: 0.12675 seconds.
# ** compute cells windings time: 0.0350819 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % clear
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_100.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 52540096 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 100 iterations
# 149 function/gradient evaluations
# TLC time: 23.7628 seconds.
# arc segment time: 0.0947006 seconds.
# arc occupancy time: 25.3631 seconds.
# - arc arrangement time: 24.8664 seconds.
# ** subdivide polyArc time: 24.5758 seconds.
# ### arc intersection init time: 0.0548575 seconds.
# ### arc bbox intersection test time: 4.5138 seconds.
# ### arc-arc intersection time: 19.5127 seconds.
# ### arc subdivision time: 0.0192328 seconds.
# ** decompose into cells time: 0.111446 seconds.
# ** compute cells windings time: 0.032859 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_200.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 127588470 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 200 iterations
# 351 function/gradient evaluations
# TLC time: 57.5138 seconds.
# arc segment time: 0.378973 seconds.
# arc occupancy time: 61.8189 seconds.
# - arc arrangement time: 60.5565 seconds.
# ** subdivide polyArc time: 59.845 seconds.
# ### arc intersection init time: 0.16311 seconds.
# ### arc bbox intersection test time: 10.9358 seconds.
# ### arc-arc intersection time: 47.5578 seconds.
# ### arc subdivision time: 0.0448703 seconds.
# ** decompose into cells time: 0.296055 seconds.
# ** compute cells windings time: 0.0767681 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_300.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 181617260 [microseconds]
# result: XTOL_REACHED
# met custom stop criteria (none): no
# 280 iterations
# 512 function/gradient evaluations
# TLC time: 82.4091 seconds.
# arc segment time: 0.345221 seconds.
# arc occupancy time: 88.2465 seconds.
# - arc arrangement time: 86.494 seconds.
# ** subdivide polyArc time: 85.508 seconds.
# ### arc intersection init time: 0.176873 seconds.
# ### arc bbox intersection test time: 15.676 seconds.
# ### arc-arc intersection time: 67.8702 seconds.
# ### arc subdivision time: 0.0649848 seconds.
# ** decompose into cells time: 0.397385 seconds.
# ** compute cells windings time: 0.103522 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_400.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 177738464 [microseconds]
# result: XTOL_REACHED
# met custom stop criteria (none): no
# 280 iterations
# 512 function/gradient evaluations
# TLC time: 80.6551 seconds.
# arc segment time: 0.323519 seconds.
# arc occupancy time: 86.4378 seconds.
# - arc arrangement time: 84.7354 seconds.
# ** subdivide polyArc time: 83.743 seconds.
# ### arc intersection init time: 0.169776 seconds.
# ### arc bbox intersection test time: 15.3342 seconds.
# ### arc-arc intersection time: 66.5143 seconds.
# ### arc subdivision time: 0.073054 seconds.
# ** decompose into cells time: 0.381499 seconds.
# ** compute cells windings time: 0.102617 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_400.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 178226412 [microseconds]
# result: XTOL_REACHED
# met custom stop criteria (none): no
# 280 iterations
# 512 function/gradient evaluations
# TLC time: 80.8633 seconds.
# arc segment time: 0.345098 seconds.
# arc occupancy time: 86.637 seconds.
# - arc arrangement time: 84.8774 seconds.
# ** subdivide polyArc time: 83.9182 seconds.
# ### arc intersection init time: 0.17456 seconds.
# ### arc bbox intersection test time: 15.3901 seconds.
# ### arc-arc intersection time: 66.6555 seconds.
# ### arc subdivision time: 0.0676138 seconds.
# ** decompose into cells time: 0.378752 seconds.
# ** compute cells windings time: 0.0985496 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc.txt solver_options.txt test_230613.ply_2d_500.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 9.86707e-06
# theta: 0.1
# Time difference: 185179880 [microseconds]
# result: XTOL_REACHED
# met custom stop criteria (none): no
# 280 iterations
# 512 function/gradient evaluations
# TLC time: 84.0104 seconds.
# arc segment time: 0.34832 seconds.
# arc occupancy time: 90.1499 seconds.
# - arc arrangement time: 88.3624 seconds.
# ** subdivide polyArc time: 87.3607 seconds.
# ### arc intersection init time: 0.176226 seconds.
# ### arc bbox intersection test time: 16.0492 seconds.
# ### arc-arc intersection time: 69.352 seconds.
# ### arc subdivision time: 0.0677249 seconds.
# ** decompose into cells time: 0.405725 seconds.
# ** compute cells windings time: 0.10775 seconds.

In [ ]:
# version 2 result (100~1000)
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_100_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 35411546 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 100 iterations
# 102 function/gradient evaluations
# TLC time: 15.5125 seconds.
# arc segment time: 0.0627798 seconds.
# arc occupancy time: 17.1713 seconds.
# - arc arrangement time: 16.8433 seconds.
# ** subdivide polyArc time: 16.6577 seconds.
# ### arc intersection init time: 0.0337742 seconds.
# ### arc bbox intersection test time: 2.95312 seconds.
# ### arc-arc intersection time: 13.3383 seconds.
# ### arc subdivision time: 0.0126116 seconds.
# ** decompose into cells time: 0.0742092 seconds.
# ** compute cells windings time: 0.0195323 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_200_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 71592159 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 200 iterations
# 202 function/gradient evaluations
# TLC time: 31.166 seconds.
# arc segment time: 0.133386 seconds.
# arc occupancy time: 34.5417 seconds.
# - arc arrangement time: 33.8607 seconds.
# ** subdivide polyArc time: 33.4917 seconds.
# ### arc intersection init time: 0.0662619 seconds.
# ### arc bbox intersection test time: 5.93619 seconds.
# ### arc-arc intersection time: 26.8265 seconds.
# ### arc subdivision time: 0.0283465 seconds.
# ** decompose into cells time: 0.148073 seconds.
# ** compute cells windings time: 0.0386838 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_300_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 107062951 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 300 iterations
# 302 function/gradient evaluations
# TLC time: 46.5114 seconds.
# arc segment time: 0.190582 seconds.
# arc occupancy time: 51.4542 seconds.
# - arc arrangement time: 50.4716 seconds.
# ** subdivide polyArc time: 49.9203 seconds.
# ### arc intersection init time: 0.102369 seconds.
# ### arc bbox intersection test time: 8.86411 seconds.
# ### arc-arc intersection time: 39.9765 seconds.
# ### arc subdivision time: 0.040835 seconds.
# ** decompose into cells time: 0.219013 seconds.
# ** compute cells windings time: 0.0580955 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_400_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 152702594 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 400 iterations
# 406 function/gradient evaluations
# TLC time: 66.1472 seconds.
# arc segment time: 0.26073 seconds.
# arc occupancy time: 73.5599 seconds.
# - arc arrangement time: 72.1541 seconds.
# ** subdivide polyArc time: 71.3677 seconds.
# ### arc intersection init time: 0.151888 seconds.
# ### arc bbox intersection test time: 12.6998 seconds.
# ### arc-arc intersection time: 57.151 seconds.
# ### arc subdivision time: 0.0525579 seconds.
# ** decompose into cells time: 0.31648 seconds.
# ** compute cells windings time: 0.0812953 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_500_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 189519464 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 500 iterations
# 511 function/gradient evaluations
# TLC time: 82.2085 seconds.
# arc segment time: 0.337021 seconds.
# arc occupancy time: 91.1514 seconds.
# - arc arrangement time: 89.4039 seconds.
# ** subdivide polyArc time: 88.421 seconds.
# ### arc intersection init time: 0.173125 seconds.
# ### arc bbox intersection test time: 15.6834 seconds.
# ### arc-arc intersection time: 70.8201 seconds.
# ### arc subdivision time: 0.0731261 seconds.
# ** decompose into cells time: 0.38596 seconds.
# ** compute cells windings time: 0.101702 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_600_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 227156332 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 600 iterations
# 617 function/gradient evaluations
# TLC time: 98.6499 seconds.
# arc segment time: 0.398265 seconds.
# arc occupancy time: 109.055 seconds.
# - arc arrangement time: 106.968 seconds.
# ** subdivide polyArc time: 105.788 seconds.
# ### arc intersection init time: 0.21498 seconds.
# ### arc bbox intersection test time: 18.7564 seconds.
# ### arc-arc intersection time: 84.7581 seconds.
# ### arc subdivision time: 0.0867372 seconds.
# ** decompose into cells time: 0.472644 seconds.
# ** compute cells windings time: 0.125382 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_700_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 262764386 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 700 iterations
# 722 function/gradient evaluations
# TLC time: 114.035 seconds.
# arc segment time: 0.473244 seconds.
# arc occupancy time: 126.188 seconds.
# - arc arrangement time: 123.781 seconds.
# ** subdivide polyArc time: 122.401 seconds.
# ### arc intersection init time: 0.242549 seconds.
# ### arc bbox intersection test time: 21.6974 seconds.
# ### arc-arc intersection time: 98.1084 seconds.
# ### arc subdivision time: 0.0918968 seconds.
# ** decompose into cells time: 0.534391 seconds.
# ** compute cells windings time: 0.146775 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_800_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 299070643 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 800 iterations
# 828 function/gradient evaluations
# TLC time: 129.35 seconds.
# arc segment time: 0.529333 seconds.
# arc occupancy time: 143.901 seconds.
# - arc arrangement time: 141.204 seconds.
# ** subdivide polyArc time: 139.656 seconds.
# ### arc intersection init time: 0.278484 seconds.
# ### arc bbox intersection test time: 24.84 seconds.
# ### arc-arc intersection time: 111.8 seconds.
# ### arc subdivision time: 0.106963 seconds.
# ** decompose into cells time: 0.622988 seconds.
# ** compute cells windings time: 0.159932 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_900_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 329756021 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 900 iterations
# 932 function/gradient evaluations
# TLC time: 142.703 seconds.
# arc segment time: 0.579324 seconds.
# arc occupancy time: 158.619 seconds.
# - arc arrangement time: 155.585 seconds.
# ** subdivide polyArc time: 153.886 seconds.
# ### arc intersection init time: 0.308386 seconds.
# ### arc bbox intersection test time: 27.2252 seconds.
# ### arc-arc intersection time: 123.273 seconds.
# ### arc subdivision time: 0.124889 seconds.
# ** decompose into cells time: 0.67395 seconds.
# ** compute cells windings time: 0.181443 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_1000_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 363175144 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 1000 iterations
# 1039 function/gradient evaluations
# TLC time: 157.534 seconds.
# arc segment time: 0.644968 seconds.
# arc occupancy time: 174.41 seconds.
# - arc arrangement time: 171.092 seconds.
# ** subdivide polyArc time: 169.223 seconds.
# ### arc intersection init time: 0.338937 seconds.
# ### arc bbox intersection test time: 30.1272 seconds.
# ### arc-arc intersection time: 135.445 seconds.
# ### arc subdivision time: 0.145998 seconds.
# ** decompose into cells time: 0.745432 seconds.
# ** compute cells windings time: 0.194353 seconds.

In [ ]:
# version 2 result (1500~3000)
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_1500_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 575972878 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 1500 iterations
# 1560 function/gradient evaluations
# TLC time: 249.628 seconds.
# arc segment time: 0.99736 seconds.
# arc occupancy time: 276.668 seconds.
# - arc arrangement time: 271.275 seconds.
# ** subdivide polyArc time: 268.278 seconds.
# ### arc intersection init time: 0.531174 seconds.
# ### arc bbox intersection test time: 47.541 seconds.
# ### arc-arc intersection time: 214.965 seconds.
# ### arc subdivision time: 0.218554 seconds.
# ** decompose into cells time: 1.17954 seconds.
# ** compute cells windings time: 0.312154 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_2000_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 749292018 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 2000 iterations
# 2094 function/gradient evaluations
# TLC time: 325.325 seconds.
# arc segment time: 1.31994 seconds.
# arc occupancy time: 359.321 seconds.
# - arc arrangement time: 352.459 seconds.
# ** subdivide polyArc time: 348.566 seconds.
# ### arc intersection init time: 0.695433 seconds.
# ### arc bbox intersection test time: 61.9891 seconds.
# ### arc-arc intersection time: 278.951 seconds.
# ### arc subdivision time: 0.265609 seconds.
# ** decompose into cells time: 1.53532 seconds.
# ** compute cells windings time: 0.408237 seconds.
# (base) kronos.di.vlad@KronosdiVlad-2 cmake-build-debug % ./SEA_QN test_230613.ply_tlc_v2.txt solver_options.txt test_230613.ply_2d_3000_v2.txt
# form: harmonic
# alphaRatio: 1e-05
# alpha: 8.17472e-06
# theta: 0.1
# Time difference: 1192655918 [microseconds]
# result: MAXITER_REACHED
# met custom stop criteria (none): no
# 3000 iterations
# 3210 function/gradient evaluations
# TLC time: 499.396 seconds.
# arc segment time: 2.02869 seconds.
# arc occupancy time: 595.143 seconds.
# - arc arrangement time: 584.535 seconds.
# ** subdivide polyArc time: 578.414 seconds.
# ### arc intersection init time: 1.08898 seconds.
# ### arc bbox intersection test time: 95.1197 seconds.
# ### arc-arc intersection time: 471.665 seconds.
# ### arc subdivision time: 0.443952 seconds.
# ** decompose into cells time: 2.48199 seconds.
# ** compute cells windings time: 0.62659 seconds.


In [156]:
dictionary = read_TLC_result(flat_filename_700)
V_2d = np.zeros(V.shape)
V_2d[:,0:2] = dictionary["resV"]

flat_plot = mp.plot(v=V_2d, 
                                 f=F, 
                                 c=mesh_color)
flat_plot.add_points(
  points=V_2d[handle_ids] + [0, 0, 0.005], # <- big points: where the small points WILL BE
  c=handle_color,
  shading={"point_size": 2})
mesh_lines(vertices=V_2d, faces=F, plot=flat_plot, color="red")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5349541…

In [126]:
mover = crestline_mover( 
            mesh_3d_vertices=V, mesh_faces=F, 
            mesh_3d_crestlines_vertices=crestline_V_3d, 
            crestline_edges=crestline_E, 
            mesh_2d_vertices=V_2d, color=mesh_color)
flat_plot_lines = mp.plot(v=V_2d, f=F, c=mesh_color)

# flat_plot_lines.add_edges(vertices=mover.mesh_2d_crestlines_vertices, 
#                           edges=crestline_E, 
#                           shading={"line_color": "green"})
flat_plot_lines.add_lines(
                  beginning=mover.mesh_2d_crestlines_vertices[crestline_E[:,0]], 
                  ending=mover.mesh_2d_crestlines_vertices[crestline_E[:,1]], 
                  shading={"line_color": "black", "line_width": 1000.0})

flat_plot_lines.add_points(
  points=V_2d[handle_ids] + [0, 0, 0.005], # <- big points: where the small points WILL BE
  c=handle_color,
  shading={"point_size": 2})
mesh_lines(vertices=V_2d, faces=F, plot=flat_plot_lines, color="red")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5349544…

In [116]:
print(max(V_2d[:,0]))
print(max(mover.mesh_2d_crestlines_vertices[:,0]))
print(max(V_2d[:,1]))
print(max(mover.mesh_2d_crestlines_vertices[:,1]))

1.9301993095155727
1.9283377798560348
2.7095214780269594
2.615633848561146
